# Real-Time Analysis of Running SPI Computations

This notebook demonstrates how to analyze MTS-SPI results **while your compute job is still running**.

## Use Case
- Terminal 1: `python -m spimts compute --mode dev++ > log.txt` (running overnight)
- Terminal 2 / This Notebook: Analyze completed models immediately, no waiting!

## How It Works
Each model writes to its own folder as soon as it completes:
- `results/dev++/<timestamp>_<model>/arrays/*.npy` 
- `results/dev++/<timestamp>_<model>/csv/*.csv`

The visualize pipeline **only reads files**, never recomputes. Safe for concurrent access.

## 1. Check Which Models Have Completed

Scan the results folder to see what's available right now.

In [ ]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set profile to match your running job
PROFILE = "dev++"  # Change to "dev", "dev+", or "paper" as needed
RESULTS_DIR = Path("./results") / PROFILE

def list_completed_models():
    """List all models that have completed computation."""
    if not RESULTS_DIR.exists():
        print(f"[INFO] No results folder found at {RESULTS_DIR}")
        return []
    
    completed = []
    for folder in RESULTS_DIR.glob("*"):
        if not folder.is_dir():
            continue
        
        # Check if meta.json exists (indicates completion)
        meta_path = folder / "meta.json"
        if meta_path.exists():
            with open(meta_path) as f:
                meta = json.load(f)
            completed.append({
                'folder': folder.name,
                'model': meta.get('model', 'Unknown'),
                'run_id': meta.get('run_id', 'Unknown'),
                'M': meta.get('M'),
                'T': meta.get('T'),
                'n_spis': meta.get('n_spis'),
                'path': folder
            })
    
    return sorted(completed, key=lambda x: x['folder'])

# List what's completed so far
completed_models = list_completed_models()
print(f"[INFO] Found {len(completed_models)} completed model(s):\n")

for i, model_info in enumerate(completed_models, 1):
    print(f"{i}. {model_info['model']:20s} | M={model_info['M']:2d}, T={model_info['T']:5d} | {model_info['n_spis']} SPIs")
    print(f"   Run ID: {model_info['run_id']}")
    print()

## 2. Load Results from a Specific Completed Model

Pick any completed model and load its artifacts (timeseries, MPI matrices, off-diagonals).

In [ ]:
# Select a model by index (0-based)
MODEL_IDX = 0  # Change this to select different models

if not completed_models:
    print("[WARN] No completed models found. Run the compute pipeline first!")
else:
    selected = completed_models[MODEL_IDX]
    model_path = selected['path']
    arrays_dir = model_path / "arrays"
    
    print(f"[INFO] Loading: {selected['model']} (M={selected['M']}, T={selected['T']})\n")
    
    # Load timeseries
    timeseries_path = arrays_dir / "timeseries.npy"
    X = np.load(timeseries_path) if timeseries_path.exists() else None
    
    # Load all MPI matrices
    matrices = {}
    offdiags = {}
    
    for npy_file in arrays_dir.glob("mpi_*.npy"):
        spi_name = npy_file.stem.replace("mpi_", "")
        matrices[spi_name] = np.load(npy_file)
    
    for npy_file in arrays_dir.glob("offdiag_*.npy"):
        spi_name = npy_file.stem.replace("offdiag_", "")
        offdiags[spi_name] = np.load(npy_file)
    
    print(f"Loaded {len(matrices)} MPI matrices and {len(offdiags)} off-diagonal vectors")
    print(f"Timeseries shape: {X.shape if X is not None else 'N/A'}")
    print(f"\nAvailable SPIs: {list(matrices.keys())[:10]}{'...' if len(matrices) > 10 else ''}")

## 3. Quick Visualization: Plot MPI Heatmaps

Visualize the interaction matrices for selected SPIs.

In [ ]:
if matrices:
    spis_to_plot = list(matrices.keys())[:6]
    fig, axes = plt.subplots(2, 3, figsize=(14, 9), dpi=120)
    axes = axes.flatten()
    
    for i, spi in enumerate(spis_to_plot):
        mat = matrices[spi]
        lower_zero = spi.lower().startswith(("mi_", "tlmi_", "te_"))
        vmin = 0.0 if lower_zero else -1.0
        vmax = None if lower_zero else 1.0
        center = None if lower_zero else 0.0
        
        sns.heatmap(mat, vmin=vmin, vmax=vmax, center=center, cmap="icefire",
                    annot=False, square=True, xticklabels=False, yticklabels=False,
                    cbar=True, cbar_kws={"shrink": .7}, linewidths=.3, ax=axes[i])
        axes[i].set_title(spi, fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("[WARN] No matrices loaded yet!")

## 4. Custom Analysis: SPI Correlations

In [ ]:
from scipy.stats import spearmanr

if offdiags:
    subset_spis = list(offdiags.keys())[:10]
    n = len(subset_spis)
    corr_matrix = np.eye(n)
    
    for i in range(n):
        for j in range(i+1, n):
            x, y = offdiags[subset_spis[i]], offdiags[subset_spis[j]]
            if np.std(x) > 0 and np.std(y) > 0 and np.all(np.isfinite(x)) and np.all(np.isfinite(y)):
                r, _ = spearmanr(x, y)
                corr_matrix[i, j] = corr_matrix[j, i] = r
    
    plt.figure(figsize=(10, 8), dpi=120)
    sns.heatmap(corr_matrix, vmin=-1, vmax=1, center=0, cmap="coolwarm",
                xticklabels=subset_spis, yticklabels=subset_spis,
                annot=True, fmt=".2f", square=True)
    plt.title(f"SPI Correlations: {selected['model']}")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 5. Use Visualize Pipeline (Recommended)

The built-in visualize command works on completed models immediately.

In [ ]:
# From terminal or this notebook using !:
# !python -m spimts visualize --profile dev++ --all-runs

print("Run these commands in a terminal while compute is running:")
print("  python -m spimts visualize --profile dev++ --all-runs")
print("  python -m spimts visualize --profile dev++ --models \"VAR(1)\"")
print("\nOutput: results/dev++/<run_id>_<model>/plots/")